###Create animated choropleth map of the temperature of countries over time.

In [34]:
import pandas as pd
import numpy as np
import plotly.express as px
from sqlalchemy import create_engine
from sqlalchemy import text
import plotly.graph_objects as go
import plotly.figure_factory as ff



In [35]:
from dotenv import dotenv_values

config = dotenv_values()


In [36]:
username = config['USER']
password = config['PASS']
host = config['HOST']
port = config['PORT']

In [37]:



url = f'postgresql://{username}:{password}@{host}:{port}/climate'

In [38]:
engine = create_engine(url, echo=False)

In [ ]:
query = """
   SELECT ymt.year, ymt.yearly_temp AS average_temp, s.cn, s.staname
FROM yearly_mean_temperature ymt
JOIN stations s ON ymt.staid = s.staid;

"""

with engine.begin() as conn:
    df = pd.read_sql(query, conn)


fig = px.choropleth(
    data_frame=df,
    locations="cn",       # Country alpha codes column
    locationmode="ISO-3", # Specify the location mode for alpha codes
    color="average_temp", # Column for coloring the map (average temperature)
    hover_name="staname", # Column to add to hover information (country name)
    animation_frame="year", # Column to animate the plot (year)
    color_continuous_scale="Viridis", # Choose a color scale
    projection="natural earth",      # Choose a map projection
    title="Yearly Average Temperature per Country", # Plot title
)

fig.show()

In [ ]:

# Assuming you have already created the 'engine' object with appropriate database connection details


query = """
   SELECT s.staname AS station_name, s.cn AS alpha_code, s.lat AS latitude, s.lon AS longitude
FROM stations s
INNER JOIN mean_temperature mt ON s.staid = mt.staid;

"""


# with engine.begin() as conn:
#     df = pd.read_sql(query, conn)


fig = go.Figure(data=go.Scattergeo(
    lon = df['longitude'],
    lat = df['latitude'],
    text = df['station_name'] + ', ' + df['alpha_code'],
    mode = 'markers',
    marker_color = 'blue',
))

fig.update_layout(
    title = 'Weather Stations Scattermap',
    geo_scope='world', # Set the scope of the map
)


fig.show()